In [77]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn.linear_model import LinearRegression #線型回帰
from sklearn.svm import SVC #サポートベクターマシン
from sklearn.ensemble import RandomForestClassifier #ランダムフォレスト
from sklearn.metrics import accuracy_score  #機械学習モデルの性能評価のためのモジュール

In [139]:
#定数を定義
BINS = 4000  #ヒストグラムのビンの数
EPSILON = .00001  #スムージングパラメータ
UPPER_LIMIT = 1.1 #静止区間の上限
LOWER_LIMIT = 0.9 #静止区間の加減
STATIONARY_INTERVALS = 5  #静止区間除去のサンプルの間隔(静止区間が何サンプル連続したら除去するか)
TRAIN_SIZE = 0.8  #ランダムフォレストのトレーニングデータの割合

In [79]:
#ディレクトリ内のデータセットのファイル名と周波数を取得する関数
def get_filename_and_Hz(path: str) -> list[int, str]:
    filename = os.listdir(path) #引数のパスのディレクトリの中のファイル名一覧を取得
    filename_and_Hz=[]  #ファイル名と周波数を格納するリストを宣言

    for file in filename:
        Hz = re.search(r'\d+', file)    #正規表現を用いてファイル名の中で一番最初に出てくる数字(周波数)を取得
        if Hz:  #数字の入っていないファイル名があるとエラーを吐くので、このif文でチェックする
            filename_and_Hz.append([int(Hz.group(0)), file])    #ファイル名と周波数を格納

    return filename_and_Hz

In [80]:
#ファイル名と周波数を分けて出力する関数
def divide_filename_and_Hz(filename_and_Hz: list[int, str]) -> tuple[list[int], list[str]]:
    Hz = []
    filename = []
    for row in filename_and_Hz:
      Hz.append(row[0])
      filename.append(row[1])

    return Hz, filename

In [81]:
#加速度データのCSVファイルから3軸加速度を取得する関数
def get_acceleration(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[2]))
            AccY.append(float(row[3]))
            AccZ.append(float(row[4]))

    return AccX, AccY, AccZ

In [82]:
#静止区間を除去する関数
def remove_stationary_intervals(AccX: list[float], AccY: list[float], AccZ: list[float]) -> list[float]:
    #各軸の加速度の平均を求める
    AvgAccX = sum(AccX) / len(AccX)
    AvgAccY = sum(AccY) / len(AccY)
    AvgAccZ = sum(AccZ) / len(AccZ)

    AvgResultantAcc = math.sqrt(AvgAccX ** 2 + AvgAccY ** 2 + AvgAccZ ** 2) #重力加速度の推定値=合成加速度の平均を求める

    ResultantAcc = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(AccX, AccY, AccZ)]   #各時刻の合成加速度を求める

    #各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が一定以上以上連続している区間)を除去する
    i=0 #ループ変数
    counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
    while i < len(ResultantAcc):
        if AvgResultantAcc * LOWER_LIMIT < ResultantAcc[i] < AvgResultantAcc * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
            counter += 1    #範囲内ならカウントを増やす
            if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
                del ResultantAcc[i+1-STATIONARY_INTERVALS:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書くとstart <= x < stopの範囲が選択される #start番目の値は含まれるがstop番目の値は含まれない
                counter = 0 #カウンターをリセット
                i -= STATIONARY_INTERVALS   #削除した分インデックスがズレるので補正する
        else:
            counter = 0 #カウンターをリセット
        i += 1

    return ResultantAcc  #静止区間を除去した後のリストを返す

In [83]:
#連続する2サンプルの差分を取る関数
def differences_of_acceleration(ResultantAcc: list[float]) -> list[float]:
    index = 0
    DifferenceAcc = []
    for dif in ResultantAcc[:-1]:
        DifferenceAcc.append(math.fabs(ResultantAcc[index + 1]*100000 - ResultantAcc[index]*100000))    #100000倍して誤差を取る
        index += 1

    return DifferenceAcc

In [84]:
#KLダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def KL_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #KLダイバージェンスの値を返す
    return np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, b_hist)])

In [85]:
#JSダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def JS_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #2つの分布の平均値を求める
    mean_hist = (a_hist + b_hist) / 2.0

    #平均とそれぞれの分布のKLダイバージェンスを算出
    kl_a = np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, mean_hist)])
    kl_b = np.sum([ai * np.log(ai / bi) for ai, bi in zip(b_hist, mean_hist)])

    #JSダイバージェンスの値を返す
    return (kl_a + kl_b) / 2.0

In [86]:
#データフレームの各行の中で2番目に小さい値が格納されている場所を調べる関数(最小値は同じ確率分布同士の0.0)
def get_index_and_columns_of_second_smallest(df: pd.DataFrame) -> list[str, str]:
    index_and_columns_of_second_smallest = []  #データフレームの中で2番目に小さい値が格納されている場所のインデックス名とカラム名を格納する変数
    for i in range(len(df)):
        sorted_row = df.iloc[i].sort_values()   #.ilocでデータフレームの要素を行、列の番号の添字で指定する    #各行の要素を昇順に並び替える
        second_smallest_columns = sorted_row.index[1] #各行の2番目に小さい値が格納されているカラム[1]の名前を取得
        #second_smallest_label = df.columns.get_loc(second_smallest_index)
        index_and_columns_of_second_smallest.append((df.index[i], second_smallest_columns))    #インデックスとカラムのラベル名の組を二次元配列に追加
    return index_and_columns_of_second_smallest

In [144]:
#推定精度を算出する巻数
def calculate_accuracy(index_and_columns_of_second_smallest: list[str, str]) -> tuple[float, list[int]]:
    counter = 0
    error_index_list = []
    for i in range(len(index_and_columns_of_second_smallest)):
        #インデックスとカラムのラベル名が同じならばカウンターを1増やす
        if index_and_columns_of_second_smallest[i][0] == index_and_columns_of_second_smallest[i][1]:
            counter += 1
        else:
            error_index_list.append(i)
            print(f"間違ってるやつは{i}番目の{index_and_columns_of_second_smallest[i][0]}と{index_and_columns_of_second_smallest[i][1]}です")

    return (counter / len(index_and_columns_of_second_smallest)) * 100, error_index_list  #精度を100分率で返す

In [88]:
#入力された加速度の差分のリストからヒストグラムを作る関数
def create_histogram(DifferenceAcc_list: list[float]) -> np.histogram:
    min_value = min(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も小さい数
    max_value = max(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も大きい数

    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)
    print(DifferenceAcc_hist[0])

    for i in range(len(DifferenceAcc_list)):
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
        DifferenceAcc_hist[i] = (DifferenceAcc_hist[i] + EPSILON) / DifferenceAcc_hist[i].sum()     #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    return DifferenceAcc_hist

In [147]:
#KLダイバージェンスとJSダイバージェンス算出の一連の流れを自動化した関数
def KL_and_JS(path: str):
    filename_and_Hz = get_filename_and_Hz(path)
    filename_and_Hz.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_filename_and_Hz(filename_and_Hz)
    Hz = [str(hz) + "Hz" for hz in Hz]  #周波数の値+"Hz"のリストを作りデータフレームのラベルに用いる

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []
    resultKLD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    resultJSD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    error_index_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

    #KLダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultKLD[i][j] = KL_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #JSダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultJSD[i][j] = JS_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #結果を出力
    df_KLD = pd.DataFrame(resultKLD, index=Hz, columns=Hz)
    display(df_KLD)
    accuracyKLD, error_index_list = calculate_accuracy(get_index_and_columns_of_second_smallest(df_KLD))
    for i in range(len(error_index_list)):
        print(filename[error_index_list[i]])
    print(f"KLダイバージェンスによる推定精度は{accuracyKLD}%です")

    df_JSD = pd.DataFrame(resultJSD, index=Hz, columns=Hz)
    display(df_JSD)
    accuracyJSD, error_index_list = calculate_accuracy(get_index_and_columns_of_second_smallest(df_JSD))
    for i in range(len(error_index_list)):
        print(filename[error_index_list[i]])
    print(f"JSダイバージェンスによる推定精度は{accuracyJSD}%です")

In [90]:
#ランダムフォレストによる機械学習モデル構築と性能評価までを自動化した関数
def random_forest(path: str):
    filename_and_Hz = get_filename_and_Hz(path)
    filename_and_Hz.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_filename_and_Hz(filename_and_Hz)

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

    DifferenceAcc_hist = create_histogram(DifferenceAcc_list)
    x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)

    # 学習する
    clf = RandomForestClassifier(random_state=1234)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("正解率 = ", accuracy_score(y_test, y_pred))

In [114]:
path = "all_walk_data/"

In [148]:
KL_and_JS(path)

,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,...,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.098418,0.449476,0.217736,0.074731,0.130470,0.394060,0.381124,0.435622,0.382961,...,0.163438,1.117954,1.003363,1.057797,1.358448,0.413777,0.240882,1.184499,1.099950,0.990794
100Hz,0.126542,0.000000,0.197061,0.253758,0.162253,0.048508,0.178786,0.329907,0.376249,0.311677,...,0.141243,1.372271,1.217949,1.141483,1.631394,0.467753,0.200180,1.489936,1.431383,1.234702
100Hz,0.478313,0.189673,0.000000,0.376733,0.528588,0.120096,0.022417,0.392323,0.401957,0.357885,...,0.293864,1.952861,1.759122,1.396168,2.256851,0.754926,0.304384,2.160083,2.040851,1.803532
100Hz,0.101695,0.099242,0.333860,0.000000,0.230868,0.105292,0.279994,0.082317,0.083900,0.072255,...,0.282572,1.527143,1.374596,1.387711,1.778540,0.645740,0.384565,1.644740,1.543022,1.388966
100Hz,0.276315,0.315948,0.638156,0.879652,0.000000,0.295725,0.653475,1.382995,1.324143,1.108790,...,0.115507,0.783034,0.689549,0.716141,0.989115,0.223728,0.147624,0.851107,0.803566,0.668632
100Hz,0.184853,0.068711,0.151655,0.220897,0.219445,0.000000,0.131191,0.329507,0.333313,0.280070,...,0.150911,1.466727,1.292740,1.147924,1.720330,0.506138,0.198784,1.610119,1.499466,1.327210
100Hz,0.422843,0.158694,0.014532,0.313086,0.485397,0.091698,0.000000,0.325748,0.337493,0.304934,...,0.272912,1.904686,1.713161,1.365878,2.201272,0.723901,0.282607,2.100192,1.976789,1.747773
100Hz,0.175409,0.105101,0.294404,0.052723,0.324791,0.129106,0.251287,0.000000,0.048996,0.018997,...,0.327907,1.814178,1.531785,1.534827,1.960854,0.744158,0.459815,1.826835,3.227368,1.574762
100Hz,0.185263,0.125550,0.317947,0.054228,0.333518,0.142133,0.272651,0.035188,0.000000,0.019178,...,0.347571,1.832080,1.547356,1.554087,1.968805,0.762975,0.475151,1.843851,2.686644,1.590025
100Hz,0.195215,0.128377,0.310038,0.051970,0.343350,0.140693,0.265733,0.025877,0.022332,0.000000,...,0.351302,1.748534,1.559848,1.558971,1.990660,0.772316,0.477429,1.864837,1.832801,1.595534


間違ってるやつは0番目の100Hzと50Hzです
間違ってるやつは4番目の100Hzと50Hzです
間違ってるやつは5番目の100Hzと50Hzです
間違ってるやつは13番目の50Hzと100Hzです
間違ってるやつは17番目の50Hzと100Hzです
間違ってるやつは22番目の50Hzと100Hzです
間違ってるやつは23番目の50Hzと100Hzです
間違ってるやつは25番目の10Hzと50Hzです
walk100Hz-20230303-111623870.csv
walk100Hz-20230228-195310844.csv
walk100Hz-20230227-202328538.csv
walk50Hz-20230302-165451196.csv
walk50Hz-20230227-202335920.csv
walk50Hz-0803-1229.csv
walk50Hz-0803-1126.csv
walk10Hz-20230302-165445244.csv
KLダイバージェンスによる推定精度は77.77777777777779%です


,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,...,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.021898,0.106764,0.028098,0.021601,0.032542,0.094281,0.049189,0.053343,0.056771,...,0.043880,0.281697,0.261853,0.272265,0.328002,0.118394,0.065006,0.284217,0.268780,0.253218
100Hz,0.021898,0.000000,0.043235,0.025650,0.040395,0.007200,0.036636,0.029013,0.034495,0.035773,...,0.034420,0.313216,0.291156,0.283192,0.356307,0.128264,0.050924,0.322395,0.309440,0.286711
100Hz,0.106764,0.043235,0.000000,0.079975,0.126708,0.028227,0.002163,0.073184,0.077714,0.075222,...,0.078168,0.396308,0.372568,0.339145,0.433471,0.197304,0.083743,0.413339,0.398557,0.375497
100Hz,0.028098,0.025650,0.079975,0.000000,0.069443,0.027307,0.068123,0.013339,0.012734,0.012310,...,0.081240,0.374469,0.349618,0.348779,0.416196,0.185509,0.107451,0.390373,0.370943,0.349595
100Hz,0.021601,0.040395,0.126708,0.069443,0.000000,0.051540,0.117584,0.093716,0.097762,0.101749,...,0.019901,0.185009,0.173153,0.179595,0.231133,0.056442,0.031026,0.185929,0.175537,0.157802
100Hz,0.032542,0.007200,0.028227,0.027307,0.051540,0.000000,0.022081,0.034301,0.037952,0.037952,...,0.036949,0.326663,0.302449,0.288085,0.367188,0.137414,0.050161,0.339853,0.323378,0.302037
100Hz,0.094281,0.036636,0.002163,0.068123,0.117584,0.022081,0.000000,0.063487,0.067892,0.065766,...,0.074409,0.393962,0.370008,0.338765,0.432250,0.193277,0.080092,0.410626,0.394766,0.372396
100Hz,0.049189,0.029013,0.073184,0.013339,0.093716,0.034301,0.063487,0.000000,0.005510,0.004159,...,0.093009,0.404130,0.373671,0.369929,0.438614,0.207038,0.124776,0.415469,0.417913,0.377335
100Hz,0.053343,0.034495,0.077714,0.012734,0.097762,0.037952,0.067892,0.005510,0.000000,0.002654,...,0.099037,0.410609,0.379956,0.375894,0.443782,0.213039,0.129632,0.422764,0.419149,0.384946
100Hz,0.056771,0.035773,0.075222,0.012310,0.101749,0.037952,0.065766,0.004159,0.002654,0.000000,...,0.101121,0.411590,0.384533,0.379628,0.449249,0.217454,0.131809,0.429089,0.413213,0.388428


間違ってるやつは0番目の100Hzと50Hzです
間違ってるやつは4番目の100Hzと50Hzです
間違ってるやつは13番目の50Hzと100Hzです
間違ってるやつは25番目の10Hzと50Hzです
間違ってるやつは31番目の10Hzと50Hzです
walk100Hz-20230303-111623870.csv
walk100Hz-20230228-195310844.csv
walk50Hz-20230302-165451196.csv
walk10Hz-20230302-165445244.csv
walk10Hz-20230227-202332703.csv
JSダイバージェンスによる推定精度は86.11111111111111%です


In [142]:
random_forest(path)

[0. 0. 0. ... 0. 0. 0.]
正解率 =  0.625
